In [68]:
import matplotlib.pyplot as plt
import sys,os,random,copy
import tempfile
from os import system
import subprocess
import numpy as np
import pandas as pd



NN_bin = './disembl'
SG_bin = './sav_gol'


def JensenNet(sequence):
    outFile = tempfile.mktemp()
    inFile = tempfile.mktemp()

    open(inFile, 'w').write(sequence + '\n')
    # inFile = 'OC1.txt'
    # outFile ='OC2.txt'
    system(NN_bin + '< ' + inFile + ' >' + outFile)
    REM465 = []
    COILS = []
    HOTLOOPS = []
    resultsFile = open(outFile, 'r')
    results = resultsFile.readlines()
    resultsFile.close()
    for result in results:
        result = result.split()
        coil = round(float(result[0]), 6)
        COILS.append(coil)
        hotloop = round(float(result[1]), 6)
        HOTLOOPS.append(hotloop)
        rem465 = round(float(result[2]), 6)
        REM465.append(rem465)
    os.remove(inFile)
    os.remove(outFile)
    return COILS, HOTLOOPS, REM465


def new_smooth(window, derivative, predicted_raw):
    if len(predicted_raw) < 2 * window:
        window = len(predicted_raw) / 2
    elif window == 0:
        window = 1
    outFile = tempfile.mktemp()
    inFile = tempfile.mktemp()
    # inFile = 'oc1_test.txt'
    # outFile ='oc1_test_result.txt'
    with open(inFile, 'w') as filehandle:
        for e in predicted_raw:
            filehandle.write('%s\n' % e)
    system_call = SG_bin + ' -V0 -D' + str(derivative) + ' -n' + str(window) + ',' + str(
        window) + ' ' + inFile + ' >' + outFile

    system(system_call)
    SG_results = []
    resultsFile = open(outFile, 'r')
    results = resultsFile.readlines()
    resultsFile.close()

    for result in results:
        temp = round(float(result), 6)
        SG_results.append(temp)
    os.remove(outFile)
    os.remove(inFile)

    return SG_results


def generate_parents(sequence, num_parents=50,parents = set()):  # by default get 50 seq as the parents
    """generate parents for the GA by random shuffling the seq"""
    count = 0
    if not sequence:
        print('you do not have input')
        return
    while True:
        temp = list(sequence)
        random.shuffle(temp)
        temp = ''.join(temp)
        if temp not in parents:
            parents.add(temp)
            count += 1
        if count == num_parents:
            break
    return list(parents)


def point_transpose(seq, position):
    """given a position, we move the amino acid in that position to a random place"""
    if not seq or position < 0:
        print('illegal input, seq empty or position is minus')
        return
    temp = list(seq) # so We will working on temp, seq is not affected
    good_mutation = False
    all_pos = list(range(len(temp)))  # all positions from 1,2,3... end of the list
    all_pos.remove(position)  # remove the position from which we pop the amino acide
    amino_acid = temp.pop(position)  # pop the amino acid at the position

    if not good_mutation:
        new_pos = random.choice(all_pos)  # choose new pos by random
        temp.insert(new_pos, amino_acid)  # insert the aa into the new postion
        temp = ''.join(temp)
        if temp != seq:
            good_mutation = True
    # do we want to have a new one?
    return temp


def frag_transpose(seq, start, stop, to_shuffle):
    if not seq or (start < 0) or (stop - start <= 0) or stop >= len(
            seq):  # I do not want to use index more that the range
        print('illegal input, seq empty or start and stop positions are wrong')
        return

    temp = list(seq)
    good_mutation = False
    try:
        frag = temp[start: stop + 1]  # pop from temp
        left_over = temp[:start] + temp[stop + 1:]
    except:
        print("""cannot retrive the frag,the length of the original list is {}, 
                your start is {}, and your stop is: {} """.format(len(temp), start, stop))
        return

    all_pos = list(range(len(left_over)))
    all_pos.remove(start)  # we do not want to be in the same
    while not good_mutation:
        new_start = random.choice(all_pos)
        if to_shuffle:
            random.shuffle(frag)
        # print(frag)
        new_seq = left_over[:new_start] + frag + left_over[new_start:]
        new_seq = ''.join(new_seq)
        if new_seq != seq:
            good_mutation = True
    return new_seq


def frag_disperse(seq, start, stop):
    """ this one we will just disperse the orginal frag into a new list

    """

    if not seq or (start < 0) or (stop - start <= 0) or stop >= len(
            seq):  # I do not want to use index more that the range
        print('illegal input, seq empty or start and stop positions are wrong')
        return

    good_mutation = False
    temp = list(seq) # a new oject so the original one is not affected
    frag = temp[start:stop + 1]
    random.shuffle(frag)
    left_over = temp[:start] + temp[stop + 1:]
    all_pos = list(range(len(left_over)))

    while not good_mutation:
        pos_needed = random.sample(all_pos, k=stop - start + 1)
        pos_needed.sort(reverse=True)  # sort it so I can insert from the revese manner
        #print(pos_needed)
        for idx, element in zip(pos_needed, frag):
            left_over.insert(idx, element)
        left_over = ''.join(left_over)
        if left_over != seq:
            good_mutation = True

    return left_over


def fitness_one(seq, to_plot=False):
    smooth_frame = 8
    _, _, raw = JensenNet(seq)
    temp_smooth = new_smooth(smooth_frame, 0, raw)
    if to_plot:
        plt.plot(temp_smooth)
        plt.axhline(y=0.5)
        plt.axvline(x=np.argmax(temp_smooth))
        plt.grid()
        plt.title('Disorder of ' + seq)
        plt.text(0, 0.35, 'mean disorder: {}'.format(round(np.mean(temp_smooth), 3)))
        plt.text(np.argmax(temp_smooth) + 1, max(temp_smooth) - 0.05, 'max:{}'.format(round(max(temp_smooth), 3)))
    return seq, np.mean(temp_smooth), np.argmax(temp_smooth), max(temp_smooth)


def scan_population(seqs):  # return the sorted one and return the survial probability
    mean_fit, max_idx, max_fit = [], [], []
    for seq in seqs:
        _, one_mean_fit, one_max_idx, one_max_fit = fitness_one(seq)
        mean_fit.append(one_mean_fit)
        max_idx.append(one_max_idx)
        max_fit.append(one_max_fit)

    prob = np.array(mean_fit)  # convert to numpy array
    idx = np.argsort(prob)  # get the rank the smallest one is in the top
    prob = 1 - prob[idx]  # sort probality and 1- so we reverse it the disorder score is 0-1
    prob = prob / prob.sum()  # survival probality we want to select the small value so 1-p

    seqs = [seqs[i] for i in idx]  # now everything is sorted based on the probability
    mean_fit = [mean_fit[i] for i in idx]
    max_idx = [max_idx[i] for i in idx]
    max_fit = [max_fit[i] for i in idx]
    # return prob as np.array, others are list
    return seqs, mean_fit, max_idx, max_fit, prob


def show_generation(seqs, mean_fit, max_idx, max_fit, prob):
    df = pd.DataFrame({'orignal seq': seqs, 'mean disorder score': mean_fit, 'survive probability': prob, \
                       'max disorder aa': max_idx, 'max_disorder socre': max_fit})
    return df

def roulette_wheel_selection(rate,prob):
    """ return a list of index which can be selected"""
    nums = int(rate * (prob.size)) # we want to have this many
    selected_index = np.random.choice(prob.size, nums, p=prob,replace = False)
    return selected_index # return selected_index as np.array


def retrive_seq(seqs, max_idx, idx):
    seqs = np.array(seqs)
    max_idx = np.array(max_idx)
    
    seqs_new = seqs[idx]
    max_idx_new = max_idx[idx]
    # so the oringal seq is not affected we have the new object
    return seqs_new, max_idx_new # return as np.array


def batch_point_tran(seqs, max_idx,ratio):  # we only need the original seq and where to generate
    # idx is the idx we should select
    # finish selection now call for mutation
    point_mutate = copy.deepcopy(seqs)
    to_mutate = int(len(point_mutate)*ratio)
    idx_to_mutate = np.random.choice(len(point_mutate), to_mutate,replace = False)
    
    for idx in idx_to_mutate:
        find = False
        while not find:
            result = point_transpose(point_mutate[idx], max_idx[idx])
            if result not in point_mutate:
                find = True
                point_mutate[idx] =result
                
    return list(point_mutate)


def correct_index(idx, nums, boundary):
    diff = nums // 2
    start = idx - diff
    stop = idx + diff

    if start < 0:
        stop = nums - 1
        start = 0

    if stop > boundary - 1:
        start = boundary - nums + 1
        stop = boundary - 1

    return start, stop


def batch_frag_disp(seqs, max_idx,nums):
    frag_disp=[]
    boundary = len(seqs[0])
    for (seq, pos) in zip(seqs,max_idx):
        start,stop =correct_index(pos,nums,boundary)
        frag_disp.append(frag_disperse(seq,start,stop))
    return frag_disp

In [108]:
def evolution(seed, total=1000, generation=20,keep_top = 50, point_ratio1 =0.5, \
              point_ratio2=0.8, frag_ratio = 0.4,frag_size =5, min_foreign = 5 ):
    
    label ='generation'
    test_seq = generate_parents(seed, num_parents= total,parents = set())
    seqs, mean_fit, max_idx, max_fit, prob = scan_population(test_seq)
    t = show_generation(seqs, mean_fit, max_idx, max_fit, prob)
    t[label] = -1
    best = t.head(2)
    for i in range(generation):
        idx = roulette_wheel_selection(point_ratio1,prob)  # 50% for point mutation
        new_seq,new_max_idx = retrive_seq(seqs, max_idx, idx)      
        point_seq = batch_point_tran(new_seq, new_max_idx,point_ratio2)   

        idx = roulette_wheel_selection(frag_ratio,prob)  
        new_seq,new_max_idx = retrive_seq(seqs, max_idx, idx)
        frag_seq= batch_frag_disp(new_seq,new_max_idx,frag_size)

        new_pop = set(point_seq + frag_seq) 
        foreign_num = total-len(new_pop)
        foreign_num = foreign_num if foreign_num >0 else min_foreign
        print('generation i introduce random', foreign_num)
        new_pop = generate_parents(seed, num_parents= foreign_num, parents= set(new_pop))
        # maintain the best 50
        top_count = 0
        for top_count in range(keep_top):
            if seqs[top_count] not in new_pop:
                new_pop.append(seqs[top_count])
        print('final number of sequence for generate{}, is {}'.format(i, len(new_pop)))
        seqs, mean_fit, max_idx, max_fit, prob = scan_population(new_pop)
        temp = show_generation(seqs, mean_fit, max_idx, max_fit, prob)
        temp[label] = i
        best = pd.concat([best,temp.head(2)],ignore_index=True)  
        if i % 10 == 0:
            display(best)
        if i % 100 == 0:
            best.to_csv('best.csv')
    return best
        
        

a = evolution('STVSDKFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDE',total=1000, point_ratio1= 0.4, generation =1000)

In [ ]:
display(a)

In [69]:

if __name__ == '__main__':
    
    label ='generation'
    seed = 'STVSDKFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDE'
    N =10
    test_seq = generate_parents(seed, num_parents= N,parents = set())
    print('orginal test seq', len(test_seq))
    seqs, mean_fit, max_idx, max_fit, prob = scan_population(test_seq)
    print('after test seq', len( seqs))
    t = show_generation(seqs, mean_fit, max_idx, max_fit, prob)
    print(t)
    t[label] = 0
    best =t.head(1)
    frag_size = 5
    for i in range(3):
        idx = roulette_wheel_selection(0.5,prob)  # 50% for point mutation

        new_seq,new_max_idx = retrive_seq(seqs, max_idx, idx)

        point_seq = batch_point_tran(new_seq, new_max_idx,0.8)   

        idx = roulette_wheel_selection(0.3,prob)  

        new_seq,new_max_idx = retrive_seq(seqs, max_idx, idx)

        frag_seq= batch_frag_disp(new_seq,new_max_idx,frag_size)
      
        
        new_pop = set(point_seq + frag_seq) 
        new_pop = generate_parents(seed, num_parents=int(0.2*N), parents= set(new_pop))
        #new_pop = new_pop + random_seq
        print('generate {}, the new pop size is {}'.format(i, len(new_pop)))
        seqs, mean_fit, max_idx, max_fit, prob = scan_population(new_pop)
        temp = show_generation(seqs, mean_fit, max_idx, max_fit, prob)
        temp[label] = i
        print(temp.head(2))
        best.append(temp.head(1))
    
    #idx = roulette_wheel_selection(0.5, e)
    
   

orginal test seq 10
after test seq 10
                                orignal seq  mean disorder score  \
0  APSHSGHHHNTLHTKHFHDMQSGSHHVEHLFHHHVHRRPD             0.588364   
1  HHVGSHQDHLPHHSDHHHKFNSRRMSTHHHVFAGLHEHPT             0.595994   
2  HHHPEHTALSHFDQHGHGHHRHSFMPSHVTKNSVRHLDHH             0.601427   
3  HLRHHGHEHSMAHHSQHHVPFHNSLHHHTTHVGPDRFSDK             0.607867   
4  KTVFQAHPSHVSHDRHHHPSTHEGNFHMLHSHHHHDRLGH             0.611263   
5  HHDTHAHHEHHLPHFHKSHGTSRMPRHGDVSLHFSVQHNH             0.618224   
6  RHHFDHSTPHPSDTHAVHHRSVHFHQHHHSGHMGLNELKH             0.621249   
7  HSHHHVQSPHHDAHGHRSHSNHHLHEFHKDGRMLPFTVTH             0.629847   
8  DKFHENAHHHSLHHHVTHHSHPRHVGDPLHMFGSTSHRQH             0.630181   
9  FRLSVHMTGGADHHNSKHPVRHHHQHHHEDSHLHFHPSHT             0.666329   

   survive probability  max disorder aa  max_disorder socre  
0             0.107498                0            0.909181  
1             0.105505                0            0.911928  
2             0.104086     

In [104]:
best.to_csv('best.csv')

In [58]:
test_seq
list(np.random.choice(test_seq, 5,replace = False))

['DHHHHQGSFHPASHHEMRRTHHSVLLVHGHDTPNFSHHKH',
 'FDHEHGVTHSHLKHHRSMVPPHHQTSHHRHDFHNHGLHSA',
 'HHHDLLHPKHEDVGPVHTGHHQSRHHSMFSANHTRSHFHH',
 'HSHTVNEDMLTFHHQDHGHHSSGVRSHHHPRFPLHHKHAH',
 'RAHHHSDVPFHLNMKTSTRHPGHHGHHFHESQVDSHHLHH']

In [59]:
a =[1,2,3,4]
3 in a

True